In [105]:
import data_load as dl
import matplotlib.pyplot as plt
import numpy as np
import sklearn
%matplotlib inline

In [2]:
path = r"C:\Users\Patrick\Datasets\sr\SR"
trial_paths = dl.combine_data(path)
trials = dl.generate_trial_objects(trial_paths)
processed_trials = dl.process_trial_data(trials)

C:\Users\Patrick\Anaconda3\lib\site-packages\openpyxl\reader\worksheet.py:318: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [3]:
#Smallest motion sample is 17559
# All G samples are 3030 long
# Take 2880 points from middle of 3030
# 3030 - 2880 = 150
#150/2 = 75
#Start each (Gs and downsampled motion on point 75) then sample out 2880

In [309]:
def synch_streams(G3, G4, motion):
    #G2 = G2[75:-75]
    G3 = G3[75:-75]
    G4 = G4[75:-75]
    #G5 = G5[75:-75]
    
    new_motion = motion[0:-1:6]
    new_motion = motion[75:(75+2880)]
    
    return G3, G4, new_motion

In [47]:
# f, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(5, 1, figsize=(100,12))
# ax1.plot(G2)
# ax2.plot(G3)
# ax3.plot(G4)
# ax4.plot(G5)
# ax5.plot(new_motion)

#Full segment is now 2880 samples for all streams at 10Hz --> 288 Seconds --> 4m48s

In [453]:
def generate_segments(G3, G4, motion):
    full_run_length = 2880
    num_segments = 200
    indices = np.random.randint(0,(full_run_length - 100), size=(num_segments))
    
    images_out = []
    motion_out = []
    
    for i in indices:
        #G2_seg = G2[i:(i+100)]/np.amax(G2[i:(i+100)])
        G3_seg = G3[i:(i+100)]/np.amax(G3[i:(i+100)])
        G4_seg = G4[i:(i+100)]/np.amax(G4[i:(i+100)])
        #G5_seg = G5[i:(i+100)]/np.amax(G5[i:(i+100)])
        motion_seg = motion[i:(i+100)]/np.amax(motion[i:(i+100)])
        
        new_image = np.array([[G3_seg],[G4_seg]])
        
        images_out.append(new_image)
        
        motion_out.append(np.asarray(motion_seg))
        
    

    return images_out, motion_out

In [117]:
# f, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(5, 1, figsize=(15,12))
# ax1.plot(segments[148][0])
# ax2.plot(segments[148][1])
# ax3.plot(segments[148][2])
# ax4.plot(segments[148][3])
# ax5.plot(segments[148][4])

In [50]:
# f, (ax1, ax2, ax3, ax4, ax5) = plt.subplots(5, 1, figsize=(15,12))
# ax1.plot(segments[148][0][0:50])
# ax2.plot(segments[148][1][0:50])
# ax3.plot(segments[148][2][0:50])
# ax4.plot(segments[148][3][0:50])
# ax5.plot(segments[148][4][0:50])

In [195]:
# test_trial = processed_trials[3]

# G3_test, motion_test = synch_streams(test_trial.G3_AVG, test_trial.Motion_Conv)
# G3, motion = generate_segments(G3_test, motion_test)

In [482]:
#Start experimenting with NNet relationship between G3 and Net Motion
#Generate a bunch of G3/net motion samples:

# trial = processed_trials[5]
# G3_temp, G4_temp, motion_temp = synch_streams(trial.G3_AVG, trial.G4_AVG, trial.Motion_Conv)
# images, motion = generate_segments(G3_temp, G4_temp, motion_temp)

# b = images[43]
        

# X = np.array(X)
# y = np.array(y)

In [481]:
#b = np.array(b)

In [480]:
#b.shape

In [479]:
#b.reshape(2,100)

In [596]:
trial = processed_trials[5]
G3_temp, G4_temp, motion_temp = synch_streams(trial.G3_AVG, trial.G4_AVG, trial.Motion_Conv)
images, motion = generate_segments(G3_temp, G4_temp, motion_temp)

X = []
y = []

for image in images:
    new_image=np.array(image)
    new_image.reshape(2,100,1)
    X.append(new_image)
    
for entry in motion:
    y.append(np.array(entry))
    
X = np.array(X).reshape(200,100,2)
y = np.array(y)

In [597]:
X.shape

(200, 100, 2)

In [598]:
y.shape

(200, 100)

In [599]:
#X = np.expand_dims(X, axis=2)
X = np.expand_dims(X, axis=3)

y = np.expand_dims(y, axis=2)
y = np.expand_dims(y, axis=3)

In [600]:
y.shape

(200, 100, 1, 1)

In [601]:
X.shape

(200, 100, 2, 1)

In [602]:
from keras.models import Sequential
from keras.layers import Conv1D, ZeroPadding1D, Conv2DTranspose, UpSampling2D
from keras.layers.pooling import MaxPooling2D, MaxPooling1D

model = Sequential()
model.add(Conv2D(10, (2,2), padding='same', activation='relu', input_shape=(100,2,1)))
model.add(MaxPooling2D(2))

# model.add(Conv2D(24, (2,2), padding='same', activation='relu'))
# model.add(MaxPooling2D(1,2))

# model.add(Conv2DTranspose(1, (1,1), strides=(1,1), activation='relu'))
# model.add(UpSampling2D(size=(1,2)))

model.add(Conv2DTranspose(1, (1,1), strides=(1,1), activation='relu'))
model.add(UpSampling2D(size=(2,1)))


model.compile(optimizer='adam', loss = 'mean_squared_error', metrics = ['accuracy'])

In [603]:
model.layers

In [604]:
for layer in model.layers:
    print(layer)
    print(layer.output_shape)

(None, 100, 2, 10)
(None, 50, 1, 10)
(None, 50, 1, 1)
(None, 100, 1, 1)


In [606]:
model.fit(X,y, epochs=100)

Epoch 1/100
200/200 [==============================] - 0s 210us/step - loss: 0.0153 - acc: 0.0100
Epoch 2/100
200/200 [==============================] - 0s 190us/step - loss: 0.0153 - acc: 0.0100
Epoch 3/100
200/200 [==============================] - 0s 185us/step - loss: 0.0153 - acc: 0.0100
Epoch 4/100
200/200 [==============================] - 0s 185us/step - loss: 0.0153 - acc: 0.0100
Epoch 5/100
200/200 [==============================] - 0s 188us/step - loss: 0.0152 - acc: 0.0100
Epoch 6/100
200/200 [==============================] - 0s 180us/step - loss: 0.0152 - acc: 0.0100
Epoch 7/100
200/200 [==============================] - 0s 190us/step - loss: 0.0152 - acc: 0.0100
Epoch 8/100
200/200 [==============================] - 0s 220us/step - loss: 0.0152 - acc: 0.0100
Epoch 9/100
200/200 [==============================] - 0s 180us/step - loss: 0.0151 - acc: 0.0100
Epoch 10/100
200/200 [==============================] - 0s 185us/step - loss: 0.0152 - acc: 0.0100
Epoch 11/100
200/20

200/200 [==============================] - 0s 195us/step - loss: 0.0131 - acc: 0.0100
Epoch 84/100
200/200 [==============================] - 0s 180us/step - loss: 0.0131 - acc: 0.0100
Epoch 85/100
200/200 [==============================] - 0s 190us/step - loss: 0.0131 - acc: 0.0100
Epoch 86/100
200/200 [==============================] - 0s 180us/step - loss: 0.0131 - acc: 0.0100
Epoch 87/100
200/200 [==============================] - 0s 175us/step - loss: 0.0131 - acc: 0.0100
Epoch 88/100
200/200 [==============================] - 0s 175us/step - loss: 0.0130 - acc: 0.0100
Epoch 89/100
200/200 [==============================] - 0s 180us/step - loss: 0.0130 - acc: 0.0100
Epoch 90/100
200/200 [==============================] - 0s 180us/step - loss: 0.0131 - acc: 0.0100
Epoch 91/100
200/200 [==============================] - 0s 180us/step - loss: 0.0131 - acc: 0.0100
Epoch 92/100
200/200 [==============================] - 0s 175us/step - loss: 0.0130 - acc: 0.0100
Epoch 93/100
200/200 [=

[<tf.Variable 'conv2d_100/kernel:0' shape=(2, 2, 1, 10) dtype=float32_ref>,
 <tf.Variable 'conv2d_100/bias:0' shape=(10,) dtype=float32_ref>,
 <tf.Variable 'conv2d_transpose_51/kernel:0' shape=(1, 1, 1, 10) dtype=float32_ref>,
 <tf.Variable 'conv2d_transpose_51/bias:0' shape=(1,) dtype=float32_ref>]